In [1]:
import networkx as nx
import pandas as pd 
import numpy as np 
from networkx.algorithms import bipartite
import warnings
warnings.filterwarnings("ignore")

mov = pd.read_csv('/Users/emilyvincett/Downloads/Employee_Movie_Choices.txt')

# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(8,18))
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

In [2]:
def answer_one():
    
    B = nx.read_edgelist('/Users/emilyvincett/Downloads/Employee_Movie_Choices.txt', delimiter='\t')
    return B
answer_one()

In [21]:
def answer_two():
    c = answer_one()
    for i in c.nodes: 
        if i in employees:
            c.add_node(i, type='employees')
        if i in movies: 
            c.add_node(i,type='movies')
    return c
   
answer_two()

In [40]:
def answer_three():
    c = answer_one()
    for i in c.nodes: 
        if i in employees:
            c.add_node(i, type='employees')
        if i in movies: 
            c.add_node(i,type='movies')
    p = bipartite.weighted_projected_graph(c,employees)
    return p
answer_three()

In [128]:
def answer_four():
    rel = nx.read_edgelist('/Users/emilyvincett/Downloads/Employee_Relationships.txt', data=[('Relationship_Score', int)])
    rel_df = pd.DataFrame(rel.edges(data=True),columns=['From','To','Relationship Score'])

    no_movies = pd.DataFrame(p.edges(data=True),columns=['From','To','No_of_movies'])
    no_movies2 = no_movies.copy()
    no_movies2.rename({'From':'To','To':'From'}, axis=1,inplace=True)
    conc_ = pd.concat([no_movies,no_movies2])
    _= rel_df.merge(conc_, left_on=['From','To'], right_on=['From','To'], how='left')
    _.fillna(value=0, inplace=True)

    _['No_of_movies'] = _['No_of_movies'].apply(lambda x: x['weight'] if type(x)==dict else None)
    _['Relationship Score'] = _['Relationship Score'].apply(lambda x: x['Relationship_Score'] if type(x)== dict else None)
    _ = _.fillna(value=0)
    correlation = _.corr(method='pearson')['Relationship Score']['No_of_movies']
    return correlation

answer_four()

0.7883962221733476